# Despacho múltiplo

Neste notebook, vamos explorar o conceito de [despacho múltiplo](https://en.wikipedia.org/wiki/Multiple_dispatch), que é um recurso chave da linguagem Julia.

Para entender despacho múltiplo em Julia, vamos começar com um exemplo que nós já vimos.

Em Julia, podemos declarar funções sem informar o tipo dos argumentos da função.
Por exemplo:

In [1]:
f(x) = x^2

f (generic function with 1 method)

Então a Julia vai determinar automaticamente sobre quais tipos de argumentos faz sentido aplicar a função, e sobre quais tipos não faz:

In [2]:
f(9)

81

In [3]:
f([1, 2, 3])

LoadError: MethodError: no method matching ^(::Vector{Int64}, ::Int64)
[0mClosest candidates are:
[0m  ^([91m::Union{AbstractChar, AbstractString}[39m, ::Integer) at /usr/share/julia/base/strings/basic.jl:721
[0m  ^([91m::LinearAlgebra.Symmetric{var"#s859", S} where {var"#s859"<:Real, S<:(AbstractMatrix{<:var"#s859"})}[39m, ::Integer) at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:704
[0m  ^([91m::LinearAlgebra.Symmetric{var"#s859", S} where {var"#s859"<:Complex, S<:(AbstractMatrix{<:var"#s859"})}[39m, ::Integer) at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:705
[0m  ...

## Especificando os tipos dos argumentos

Temos a opção de declarar explicitamente quais tipos de argumentos uma função pode receber.
Por exemplo, vamos definir uma função `teste` que só pode receber argumentos do tipo `String`:

In [4]:
teste(x::String, y::String) = println("Os argumentos x e y são strings")

teste (generic function with 1 method)

Vamos verificar que a função `teste` age sobre strings, mas não age sobre argumentos de outros tipos:

In [5]:
teste("despacho", "múltiplo")

Os argumentos x e y são strings


In [6]:
teste(2, 3)

LoadError: MethodError: no method matching teste(::Int64, ::Int64)

Para fazer com que a função `teste` aceite números inteiros, vamos definir `teste` para argumentos do tipo `Int`:

In [7]:
teste(x::Int, y::Int) = println("Os argumentos x e y são números inteiros")

teste (generic function with 2 methods)

In [8]:
teste(2, 3)

Os argumentos x e y são números inteiros


Agora `teste` age sobre os inteiros.
Mas observe, a função `teste` continua definida quando `x` e `y` são strings:

In [9]:
teste("despacho", "múltiplo")

Os argumentos x e y são strings


Quando declaramos
```julia
teste(x::Int, y::Int) = println("Os argumentos x e y são números inteiros"),
```
não removemos ou substituímos
```julia
teste(x::String, y::String),
```
simplesmente adicionamos um método à função genérica `teste`.

Uma **função genérica** é um conceito abstrato associado a uma operação.
Por exemplo, a função genérica `+` representa o conceito de adição.
Um **método** é uma implementação específica de uma função genérica para um tipo específico de argumento.
Por exemplo, a função `+` possui métodos para números de precisão flutuante, inteiros, matrizes, etc.
Podemos usar `methods` para ver os métodos que a função `teste` possui:

In [10]:
methods(teste)

# 2 methods for generic function "teste":
[1] teste(x::String, y::String) in Main at In[4]:1
[2] teste(x::Int64, y::Int64) in Main at In[7]:1

E podemos ver os métodos da função `+`:

In [11]:
methods(+)

# 208 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:529
[3] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:535
[4] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:390
[5] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/periods.jl:372
[8] +(y::Dates.TimeType, x::StridedArray{<:Union{Dates.CompoundPeriod, Dates.Period}}) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/deprecated.jl:18
[9] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Tridiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/tridiag.jl:740
[10] +(x::LinearAlgebra.Tridiagonal, H::LinearAlgebra.UpperHessenberg) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/hessenberg.jl:107
[11] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:157
[12] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Diagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:173
[13] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:193
[14] +(A::LinearAlgebra.Tridiagonal{var"#s859", V} where {var"#s859"<:Number, V<:AbstractVector{var"#s859"}}, B::LinearAlgebra.UniformScaling) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:226
[15] +(index1::CartesianIndex{N}, index2::CartesianIndex{N}) where N in Base.IteratorsMD at multidimensional.jl:114
[16] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.Hermitian) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:469
[17] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.SymTridiagonal{var"#s856", V} where {var"#s856"<:Real, V<:AbstractVector{var"#s856"}}) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:488
[18] +(H::LinearAlgebra.Hermitian, D::LinearAlgebra.Diagonal{var"#s859", V} where {var"#s859"<:Real, V<:AbstractVector{var"#s859"}}) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/diagonal.jl:185
[19] +(A::LinearAlgebra.Hermitian, J::LinearAlgebra.UniformScaling{<:Complex}) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:196
[20] +(A::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /usr/share/julia/stdlib/v1.7/SparseArrays/src/linalg.jl:15
[21] +(A::LinearAlgebra.Hermitian, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /usr/share/julia/stdlib/v1.7/SparseArrays/src/linalg.jl:18
[22] +(A::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::LinearAlgebra.Symmetric{<:Real, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /usr/share/julia/stdlib/v1.7/SparseArrays/src/linalg.jl:26
[23] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.Symmetric{var"#s859", S} where {var"#s859"<:Real, S<:(AbstractMatrix{<:var"#s859"})}) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:483
[24] +(A::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::LinearAlgebra.Symmetric{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /usr/share/julia/stdlib/v1.7/SparseArrays/src/linalg.jl:24
[25] +(A::LinearAlgebra.UnitUpperTriangular, B::LinearAlgebra.UpperTriangular) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/triangular.jl:640
[26] +(A::LinearAlgebra.UnitUpperTriangular, B::LinearAlgebra.UnitUpperTriangular) in LinearAlgebra at

Portanto, agora podemos aplicar `teste` a strings e inteiros.
Quando aplicamos `teste` a um par de argumentos, a Julia vai inferir os tipos dos argumentos e despachar o método apropriado.
Isso é **despacho múltiplo**.
Despacho múltiplo torna os programas genéricos e rápidos.

Para ver qual método está sendo despachado quando chamamos uma função genérica, usamos a macro `@which`:

In [12]:
@which teste(2, 3)

teste(x::Int64, y::Int64) in Main at In[7]:1

In [13]:
@which 3.0 + 5.0

+(x::Float64, y::Float64) in Base at float.jl:399